# Contents API

## Retrieving a File's Contents

In [1]:
import github3

In [2]:
repo = github3.repository('sigmavirus24', 'github3.py')

In [3]:
contents = repo.file_contents('github3/repos/contents.py')

Now that we have the contents of the file at `github3/repos/contents.py` we can get information about it. We can find the name, the SHA of the commit that most recently updated it, etc.

In [4]:
print(contents.name)

contents.py


In [5]:
print(contents.sha)

ec9f27e0cc18a5440952ebb7dc9b3b9402b93702


We can also get the contents of the file:

In [6]:
print(contents.content)

IyAtKi0gY29kaW5nOiB1dGYtOCAtKi0KIiIiCmdpdGh1YjMucmVwb3MuY29u
dGVudHMKPT09PT09PT09PT09PT09PT09PT09PQoKVGhpcyBtb2R1bGUgY29u
dGFpbnMgdGhlIENvbnRlbnRzIG9iamVjdCBwZXJ0YWluaW5nIHRvIFJFQURN
RXMgYW5kIG90aGVyIGZpbGVzCnRoYXQgY2FuIGJlIGFjY2Vzc2VkIHZpYSB0
aGUgR2l0SHViIEFQSS4KCiIiIgpmcm9tIF9fZnV0dXJlX18gaW1wb3J0IHVu
aWNvZGVfbGl0ZXJhbHMKCmZyb20ganNvbiBpbXBvcnQgZHVtcHMKZnJvbSBi
YXNlNjQgaW1wb3J0IGI2NGRlY29kZSwgYjY0ZW5jb2RlCmZyb20gLi5naXQg
aW1wb3J0IENvbW1pdApmcm9tIC4ubW9kZWxzIGltcG9ydCBHaXRIdWJDb3Jl
CmZyb20gLi5kZWNvcmF0b3JzIGltcG9ydCByZXF1aXJlc19hdXRoCgoKY2xh
c3MgQ29udGVudHMoR2l0SHViQ29yZSk6CiAgICAiIiJUaGUgOmNsYXNzOmBD
b250ZW50cyA8Q29udGVudHM+YCBvYmplY3QuIEl0IGhvbGRzIHRoZSBpbmZv
cm1hdGlvbgogICAgY29uY2VybmluZyBhbnkgY29udGVudCBpbiBhIHJlcG9z
aXRvcnkgcmVxdWVzdGVkIHZpYSB0aGUgQVBJLgoKICAgIFR3byBjb250ZW50
IGluc3RhbmNlcyBjYW4gYmUgY2hlY2tlZCBsaWtlIHNvOjoKCiAgICAgICAg
YzEgPT0gYzIKICAgICAgICBjMSAhPSBjMgoKICAgIEFuZCBpcyBlcXVpdmFs
ZW50IHRvOjoKCiAgICAgICAgYzEuc2hhID09IGMyLnNoYQogICAgICAgIGMx
LnNoYSAhPSBjMi5zaGEKCiAg

But that's base64 encoded text. So github3.py provides the content decoded as well:

In [7]:
print(contents.decoded)

# -*- coding: utf-8 -*-
"""
github3.repos.contents

This module contains the Contents object pertaining to READMEs and other files
that can be accessed via the GitHub API.

"""
from __future__ import unicode_literals

from json import dumps
from base64 import b64decode, b64encode
from ..git import Commit
from ..models import GitHubCore
from ..decorators import requires_auth


class Contents(GitHubCore):
    """The :class:`Contents <Contents>` object. It holds the information
    concerning any content in a repository requested via the API.

    Two content instances can be checked like so::

        c1 == c2
        c1 != c2

    And is equivalent to::

        c1.sha == c2.sha
        c1.sha != c2.sha

    See also: http://developer.github.com/v3/repos/contents/
    """
    def _update_attributes(self, content):
        # links
        self._api = content.get('url')
        #: Dictionary of links
        self.links = content.get('_links')

        #: URL of the README on github.com
  

github3.py tries to preserve original names of data it receives so that users can access that easily and without having to guess. From here, (if we were logged in) we could easily update or even delete this file altogether.

## Retrieving a directory's contents

In [8]:
import github3

In [9]:
repo = github3.repository('sigmavirus24', 'github3.py')

In [10]:
dir_contents = repo.directory_contents('github3/search/')

In [11]:
dir_contents

[(u'__init__.py', <Content [github3/search/__init__.py]>),
 (u'code.py', <Content [github3/search/code.py]>),
 (u'issue.py', <Content [github3/search/issue.py]>),
 (u'repository.py', <Content [github3/search/repository.py]>),
 (u'user.py', <Content [github3/search/user.py]>)]

In [12]:
dc = dict(dir_contents)

In [13]:
dc['user.py'].decoded

u''

In many instances when GitHub returns more than one instance of a resource, they don't return the entirety of each resource. Here's we have an example of this. The file bodies would be expensive to return for a directory with a lot of files, so GitHub omits them. We can still retrieve it. We just need to make another call to the API for the information.

In [15]:
dc['user.py'].refresh()
print(dc['user.py'].decoded)

# -*- coding: utf-8 -*-
from __future__ import unicode_literals

from ..models import GitHubCore
from ..users import User


class UserSearchResult(GitHubCore):
    def _update_attributes(self, data):
        result = data.copy()
        #: Score of this search result
        self.score = result.pop('score')
        #: Text matches
        self.text_matches = result.pop('text_matches', [])
        #: User object matching the search
        self.user = User(result, self)

    def _repr(self):
        return '<UserSearchResult [{0}]>'.format(self.user)

